## Data Preparation

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vbookshelf/respiratory-sound-database")

print("Path to dataset files:", path)

/Users/shotomorisaki/Engineering/bloodCellsBioHacks/notebook/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  0%|          | 0.00/3.69G [00:00<?, ?B/s]

# EDA

In [3]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

ModuleNotFoundError: No module named 'librosa'

In [ ]:
# Function to extract spectrogram from audio
def extract_spectrogram(audio_path, n_mels=128, fmax=8000):
    try:
        y, sr = librosa.load(audio_path, sr=None)  # Load the audio file
        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, fmax=fmax)  # Compute Mel spectrogram
        log_S = librosa.power_to_db(S, ref=np.max)  # Convert to decibel scale
        return log_S
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        return None

# Function to extract MFCCs from audio
def extract_mfcc(audio_path, n_mfcc=13):
    try:
        y, sr = librosa.load(audio_path, sr=None)  # Load the audio file
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)  # Compute MFCCs
        return mfccs
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        return None

# Function to process all audio files in a given directory
def process_audio_files(dataset_path, extension='.wav', batch_size=10):
    audio_files = glob.glob(os.path.join(dataset_path, f'**/*{extension}'), recursive=True)  # Get all audio files recursively
    spectrograms = []
    mfccs = []
    
    # Process audio files in batches
    for i, audio_path in enumerate(audio_files):
        spectrogram = extract_spectrogram(audio_path)
        mfcc = extract_mfcc(audio_path)
        
        if spectrogram is not None:
            spectrograms.append(spectrogram)
        if mfcc is not None:
            mfccs.append(mfcc)
        
        # Process in batches to reduce memory usage
        if len(spectrograms) >= batch_size:
            print(f"Processed {i + 1}/{len(audio_files)} files")
            break  # Optional: Remove this line if you want to process all files
    
    # Check if any spectrograms were generated
    if not spectrograms:
        print("No spectrograms were generated.")
    
    return spectrograms, mfccs

In [ ]:
# Example usage:
dataset_path = '/root/.cache/kagglehub/datasets/vbookshelf/respiratory-sound-database/versions/2/respiratory_sound_database/Respiratory_Sound_Database/audio_and_txt_files'

# Get and print the list of files
audio_files = glob.glob(os.path.join(dataset_path, '**/*.wav'), recursive=True)
print(f"Found {len(audio_files)} audio files.")

# Process a small batch of audio files
spectrograms, mfccs = process_audio_files(dataset_path, batch_size=5)

In [ ]:
# Check if we have valid data for training
if spectrograms:
    print(f"Spectrograms shape: {np.array(spectrograms).shape}")
else:
    print("No spectrograms to process.")

# Ensure there is data to proceed with
if spectrograms:
    # Visualize a sample spectrogram if available
    plt.figure(figsize=(10, 6))
    plt.imshow(spectrograms[0], aspect='auto', origin='lower', cmap='inferno')
    plt.title('Spectrogram of First Audio File')
    plt.xlabel('Time (Frames)')
    plt.ylabel('Frequency Bins')
    plt.colorbar(format='%+2.0f dB')
    plt.show()

In [ ]:
# Check MFCCs
if mfccs:
    print(f"MFCCs shape: {np.array(mfccs).shape}")
else:
    print("No MFCCs to process.")
    
if mfccs:
    # Visualize MFCCs if available
    plt.figure(figsize=(10, 6))
    plt.imshow(mfccs[0], aspect='auto', origin='lower', cmap='viridis')
    plt.title('MFCCs of First Audio File')
    plt.xlabel('Time (Frames)')
    plt.ylabel('MFCC Coefficients')
    plt.colorbar(format='%+2.0f dB')
    plt.show()


# Model Developing

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Function to preprocess data
def preprocess_data(spectrograms, labels):
    if len(spectrograms) == 0:
        raise ValueError("No valid spectrograms to preprocess.")
    
    spectrograms = np.array(spectrograms)
    max_vals = np.max(spectrograms, axis=(1, 2), keepdims=True)
    spectrograms = spectrograms / (max_vals + 1e-10)
    
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    
    # Reshape for CNN input: (samples, height, width, channels)
    spectrograms = spectrograms.reshape(spectrograms.shape[0], spectrograms.shape[1], spectrograms.shape[2], 1)
    return spectrograms, labels

## RandomForest 

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
def extract_features(file_path, sr=22050, n_mfcc=13):
    """
    Extract audio features including MFCCs, spectral centroid,
    zero crossing rate, spectral bandwidth, and chroma features.
    """
    try:
        y, sr = librosa.load(file_path, sr=sr)
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None
    
    if y.size == 0:
        print(f"File {file_path} is empty.")
        return None

    # MFCCs
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_std = np.std(mfcc, axis=1)

    # Spectral centroid
    spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_centroid_mean = np.mean(spec_centroid)
    spec_centroid_std = np.std(spec_centroid)

    # Zero Crossing Rate
    zcr = librosa.feature.zero_crossing_rate(y)
    zcr_mean = np.mean(zcr)
    zcr_std = np.std(zcr)

    # Spectral bandwidth
    spec_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spec_bandwidth_mean = np.mean(spec_bandwidth)
    spec_bandwidth_std = np.std(spec_bandwidth)

    # Chroma features
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_mean = np.mean(chroma, axis=1)
    chroma_std = np.std(chroma, axis=1)

    # Combine features into a dictionary
    features = {}
    for i in range(n_mfcc):
        features[f'mfcc_{i+1}_mean'] = mfcc_mean[i]
        features[f'mfcc_{i+1}_std'] = mfcc_std[i]
    
    features['spec_centroid_mean'] = spec_centroid_mean
    features['spec_centroid_std'] = spec_centroid_std
    features['zcr_mean'] = zcr_mean
    features['zcr_std'] = zcr_std
    features['spec_bandwidth_mean'] = spec_bandwidth_mean
    features['spec_bandwidth_std'] = spec_bandwidth_std
    
    for i in range(chroma.shape[0]):
        features[f'chroma_{i+1}_mean'] = chroma_mean[i]
        features[f'chroma_{i+1}_std'] = chroma_std[i]
    
    return features

In [ ]:
audio_files = glob.glob(os.path.join(dataset_path, '**/*.wav'), recursive=True)
print(f"Found {len(audio_files)} audio files.")

feature_list = []
labels = []

# Loop over files and extract features with updated label extraction
for file in audio_files:
    print(f"Processing file: {file}")
    feats = extract_features(file)
    if feats is None:
        print(f"Skipping file due to extraction error: {file}")
        continue
    feature_list.append(feats)
    
    # Extract label from the file name.
    file_name = os.path.basename(file)
    parts = file_name.split('_')
    if len(parts) >= 3:
        code = parts[2]
        # Map codes to labels. Adjust these mappings as needed.
        if code in ['Ar', 'Lr']:
            label = 'healthy'
        elif code == 'Tc':
            label = 'diseased'
        else:
            label = 'unknown'
    else:
        label = 'unknown'
    
    labels.append(label)
    # print(f"Extracted label: {label}")

In [ ]:
# Create DataFrame from features and labels
df_features = pd.DataFrame(feature_list)
df_features['label'] = labels

# Optionally, filter out rows if you have unwanted labels (adjust if necessary)
# For example, if you only expect 'healthy' and 'diseased' labels:
df_features = df_features[df_features['label'].isin(['healthy', 'diseased'])]

print("Feature DataFrame shape:", df_features.shape)
print(df_features.head())

# Separate features and target labels
X = df_features.drop("label", axis=1)
y = df_features['label']

# Encode string labels to integers (binary classification)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Make predictions and evaluate the classifier
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy*100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set the style
sns.set_palette("husl")

# Create figure and axis
fig, ax = plt.subplots(figsize=(12, 6))

# Create the bar plot
bars = ax.bar(range(len(importances)), 
              importances[indices],
              color=sns.color_palette("Blues_r", len(importances)),
              align="center")

# Customize the plot
ax.set_title("Feature Importances", fontsize=14, pad=20)
ax.set_xlabel("Features", fontsize=12, labelpad=10)
ax.set_ylabel("Importance Score", fontsize=12, labelpad=10)

# Customize x-axis
ax.set_xticks(range(len(importances)))
ax.set_xticklabels(features[indices], rotation=45, ha='right')
ax.set_xlim([-0.5, len(importances)-0.5])

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    # ax.text(bar.get_x() + bar.get_width()/2., height,
    #         f'{height:.3f}',
    #         ha='center', va='bottom')

# Add grid for better readability
ax.yaxis.grid(True, linestyle='--', alpha=0.7)
ax.set_axisbelow(True)

# Adjust layout to prevent label cutoff
plt.tight_layout()

# Show plot
plt.show()